year 2024 transection

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

# ---------------------------
# Configuration and constants (updated)
# ---------------------------
ATM_ID = "ATM_013" 
AVG_DAILY_WITHDRAWALS = 27
AVG_DAILY_INQUIRIES   = 1.35

ATM_MAX_CASH = 1200000
REFILL_THRESHOLD = 10000

# Updated monthly weighting factors.
monthly_weights = {
    1: 1.1,  2: 1.0,  3: 2.7,  4: 1.6, 5: 1.0, 6: 1.4,
    7: 1.3,  8: 1.7,  9: 1.4, 10: 1.7,11: 2.9,12: 1.3
}

# Updated salary day multiplier range.
SALARY_DATE_MULTIPLIER = (1.4, 1.7)

# Updated festival spikes with multiple multipliers.  
festival_spikes = {
    "Holi":            ("2024-03-25", [1.8, 2.8, 3.2, 2.28, 1.9]),
    "Diwali":          ("2024-11-01", [2.0, 2.7, 3.5, 2.8, 2.1, 1.9, 1.6]),
    "Makar Sankranti": ("2024-01-14", [1.7]),
    "Maha Shivratri":  ("2024-03-08", [1.9]),
    "Eid-ul-Fitr":      ("2024-04-11", [2.7]),
    "Ram Navami":      ("2024-04-17", [1.9]),
    "Eid-ul-Adha":      ("2024-06-17", [1.9]),
    "Raksha Bandhan":   ("2024-08-19", [2.1]),
    "Ganesh Chaturthi": ("2024-09-07", [1.9]),
    "Dussehra":         ("2024-10-12", [2.4]),
    "Bhai Dooj":        ("2024-11-03", [2.3]),
    "Christmas":        ("2024-12-25", [1.6]),
}

# Hourly weights for 24 hours; these weights are later normalized to probabilities.
raw_hourly_weights = [
    2, 1, 0.5, 0.5, 0.5, 1, 3, 5, 6, 6,
    6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 7, 5, 4, 3
]
total_raw = sum(raw_hourly_weights)
hourly_probs = [w / total_raw for w in raw_hourly_weights]

# ---------------------------
# Sampling helper functions
# ---------------------------
def sample_refill_delay():
    """
    Return a random delay (in days) between 1 and 3.
    """
    return np.random.randint(1, 4)

def sample_cluster_delay():
    """
    Sample a delay (in minutes) for a clustered additional transaction.
    Uses an exponential distribution with a mean of 3 minutes.
    """
    delay = np.random.exponential(scale=3)
    return max(1, int(math.ceil(delay)))

def sample_withdrawal_amount(features):
    """
    Sample a withdrawal amount based on day type.
    
    • If it's a festival day → amount is uniformly chosen between 5,000 and 10,000.
    • Else if it's Saturday or Sunday → amount between 10,000 and 20,000.
    • Else if it's a salary day (and not weekend/festival) → amount between 7,000 and 20,000.
    • Otherwise (default) → amount between 10,000 and 20,000.
    
    The returned amount is rounded to the nearest 100.
    """
    if features["festival"]:
        low, high = 5000, 10000
    elif features["day_of_week"] in ("Saturday", "Sunday"):
        low, high = 10000, 20000
    elif features["salary_day"]:
        low, high = 7000, 20000
    else:
        low, high = 10000, 20000

    amount = np.random.randint(low, high + 1)
    return int(round(amount / 100.0) * 100)

# ---------------------------
# Feature engineering functions
# ---------------------------
def build_feature_vector(current_date):
    """
    Build a feature vector for the day with:
      - Festival indicator and its multiplier (if today's a festival).
      - Salary day indicator (day 1 to 5).
      - Weather conditions (rainy, coldest day, hot day) based on fixed rules.
      - Weekend flag and day-of-week name.
      - Combined event label.
    """
    features = {"date": current_date.date()}
    festival_indicator = ""
    festival_multiplier = 2.8  # default if needed
    
    # Check if today is a festival.
    for fest_name, (fest_date_str, multipliers) in festival_spikes.items():
        fest_date = datetime.strptime(fest_date_str, "%Y-%m-%d").date()
        if current_date.date() == fest_date:
            festival_indicator = fest_name
            # For multiple multipliers, we take the maximum.
            festival_multiplier = max(multipliers)
            break
    features["festival"] = festival_indicator
    features["festival_multiplier"] = festival_multiplier
    features["salary_day"] = (current_date.day in range(1, 6))
    
    weather = ""
    # Hard-coded rainy days.
    if (current_date.month, current_date.day) in {(7, 7), (8, 1), (8, 7), (8, 22), (8, 28)}:
        weather = "rainy"
    elif current_date.month == 12 and current_date.day >= 20:
        weather = "coldest day"
    # Selected hot days.
    elif (current_date.month == 6 and current_date.day in [6, 7, 8]) or (current_date.month == 7 and current_date.day in [6, 8]):
        weather = "hot day"
    features["weather"] = weather

    weekday = current_date.weekday()  # Monday=0, Sunday=6
    features["weekend"] = (weekday >= 5)
    day_of_week = current_date.strftime("%A")
    features["day_of_week"] = day_of_week

    if festival_indicator and weather:
        features["event_label"] = festival_indicator + ", " + weather
    elif festival_indicator:
        features["event_label"] = festival_indicator
    elif weather:
        features["event_label"] = weather
    else:
        features["event_label"] = ""
    
    return features

def predict_daily_demand(features):
    """
    Calculate an effective daily demand multiplier based on:
      - Festival and/or salary day (if both, multiply festival multiplier with a random salary factor).
      - Weather modifiers (rainy/cold days reduce demand; hot days reduce a bit too).
      - Weekend multipliers: Saturday ×1.3 and Sunday ×1.65.
    """
    if features["festival"] and features["salary_day"]:
        base_multiplier = features["festival_multiplier"] * np.random.uniform(SALARY_DATE_MULTIPLIER[0], SALARY_DATE_MULTIPLIER[1])
    elif features["festival"]:
        base_multiplier = features["festival_multiplier"]
    elif features["salary_day"]:
        base_multiplier = np.random.uniform(SALARY_DATE_MULTIPLIER[0], SALARY_DATE_MULTIPLIER[1])
    else:
        base_multiplier = 1.0

    if features["weather"] in ("rainy", "coldest day"):
        weather_modifier = 0.7
    elif features["weather"] == "hot day":
        weather_modifier = 0.8
    else:
        weather_modifier = 1.0

    effective_multiplier = base_multiplier * weather_modifier

    if features["day_of_week"] == "Saturday":
        effective_multiplier *= 1.3
    elif features["day_of_week"] == "Sunday":
        effective_multiplier *= 1.65

    return effective_multiplier

# ---------------------------
# Simulation state variables
# ---------------------------
remaining_cash = ATM_MAX_CASH
pending_refill_date = None  # Holds a datetime when a refill is scheduled

start_date_obj = datetime(2024, 1, 1)
end_date_obj   = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date_obj, end=end_date_obj)

all_events = []  # List to store all transaction events

# ---------------------------
# Simulation main loop: iterate day by day
# ---------------------------
for current_date in date_range:
    
    # Process any scheduled refill event if due.
    if pending_refill_date is not None and current_date.date() >= pending_refill_date.date():
        event_hour = np.random.choice(range(24), p=hourly_probs)
        minute = np.random.randint(0, 60)
        second = np.random.randint(0, 60)
        event_time = f"{event_hour:02d}:{minute:02d}:{second:02d}"
        refill_event = [
            ATM_ID,
            current_date.date(),
            current_date.strftime('%A'),
            event_time,
            "Refill",
            0,                    # No withdrawal amount
            ATM_MAX_CASH,         # Cash resets to maximum
            str(current_date.date()),  # Refill Scheduled date written here
            "Refilled",
            None                  # No festival info for refill event
        ]
        all_events.append(refill_event)
        remaining_cash = ATM_MAX_CASH
        pending_refill_date = None

    # Build daily feature vector.
    features = build_feature_vector(current_date)
    day_of_week = features["day_of_week"]
    
    # Calculate daily demand.
    effective_multiplier = predict_daily_demand(features)
    daily_weight = monthly_weights[current_date.month] * effective_multiplier
    variation_factor = np.random.uniform(0.7, 1.3)
    
    # Determine the number of withdrawals and inquiries.
    num_withdrawals = np.random.poisson(lam=AVG_DAILY_WITHDRAWALS * daily_weight * variation_factor)
    num_inquiries   = np.random.poisson(lam=AVG_DAILY_INQUIRIES * daily_weight * variation_factor)
    
    events_list = []
    
    # Helper function to generate a random event time.
    def generate_event_time():
        hour = np.random.choice(range(24), p=hourly_probs)
        minute = np.random.randint(0, 60)
        second = np.random.randint(0, 60)
        return f"{hour:02d}:{minute:02d}:{second:02d}"
    
    # Generate withdrawal events.
    for i in range(num_withdrawals):
        event_time = generate_event_time()
        amount = sample_withdrawal_amount(features)
        events_list.append({
            "time_str": event_time,
            "type": "Withdrawal",
            "amount": amount,
            "festival": features["festival"]
        })
        # Optionally produce a clustered withdrawal shortly after.
        if np.random.rand() < 0.2:
            delay_minutes = sample_cluster_delay()
            base_hour, base_minute, base_second = map(int, event_time.split(":"))
            base_total = base_hour * 60 + base_minute
            new_total = base_total + delay_minutes
            if new_total < 24 * 60:  # Ensure within same day.
                new_hour = new_total // 60
                new_minute = new_total % 60
                new_time = f"{new_hour:02d}:{new_minute:02d}:{base_second:02d}"
                events_list.append({
                    "time_str": new_time,
                    "type": "Withdrawal",
                    "amount": sample_withdrawal_amount(features),
                    "festival": features["festival"]
                })
                
    # Generate inquiry events.
    for i in range(num_inquiries):
        event_time = generate_event_time()
        events_list.append({
            "time_str": event_time,
            "type": "Inquiry",
            "amount": 0,
            "festival": features["festival"]
        })
        
    # Sort events by time so they are processed chronologically.
    events_list.sort(key=lambda x: x["time_str"])
    
    # Process each event.
    for event in events_list:
        if event["type"] == "Withdrawal":
            # In the low-cash window, all withdrawals are declined.
            if remaining_cash < REFILL_THRESHOLD:
                status = "Declined"
                if pending_refill_date is None:
                    pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                refill_info = str(pending_refill_date.date())
            else:
                if remaining_cash >= event["amount"]:
                    remaining_cash -= event["amount"]
                    status = "Completed"
                    # If cash falls below threshold, schedule a refill if none exists.
                    if remaining_cash < REFILL_THRESHOLD and pending_refill_date is None:
                        pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                        refill_info = str(pending_refill_date.date())
                    else:
                        refill_info = ""
                else:
                    status = "Declined"
                    if remaining_cash < REFILL_THRESHOLD and pending_refill_date is None:
                        pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                    refill_info = str(pending_refill_date.date()) if pending_refill_date is not None else ""
    
            processed_event = [
                ATM_ID,
                current_date.date(),
                day_of_week,
                event["time_str"],
                event["type"],
                event["amount"],
                remaining_cash,
                refill_info,  # The scheduled refill date is recorded here.
                status,
                event["festival"]
            ]
            all_events.append(processed_event)
            
        elif event["type"] == "Inquiry":
            processed_event = [
                ATM_ID,
                current_date.date(),
                day_of_week,
                event["time_str"],
                event["type"],
                0,
                remaining_cash,
                "",   # No refill info for inquiries.
                "Completed",
                event["festival"]
            ]
            all_events.append(processed_event)

# ---------------------------
# Create a DataFrame and save to CSV.
# ---------------------------
df_transactions = pd.DataFrame(all_events, columns=[
    "ATM_ID", "Date", "Day", "Time", "Transaction_Type",
    "Transaction_Amount", "Remaining_Cash", "Cash_Refill",
    "Transaction_Status", "Festival"
])
output_file = "2024ATM_0_withdrawal.csv"
df_transactions.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

Data saved to 2024ATM_099_withdrawal.csv


import pandas as pd
import os

# Creating a dictionary with festival names and their corresponding dates
festival_data = {
    "2024-03-25": "Holi",
    "2024-11-01": "Diwali",
    "2024-01-14": "Makar Sankranti",
    "2024-03-08": "Maha Shivratri",
    "2024-04-11": "Eid-ul-Fitr",
    "2024-04-17": "Ram Navami",
    "2024-06-17": "Eid-ul-Adha",
    "2024-08-19": "Raksha Bandhan",
    "2024-09-07": "Ganesh Chaturthi",
    "2024-10-12": "Dussehra",
    "2024-11-03": "Bhai Dooj",
    "2024-12-25": "Christmas",
    "2023-03-08": "Holi",
    "2023-11-12": "Diwali",
    "2023-01-14": "Makar Sankranti",
    "2023-02-18": "Maha Shivratri",
    "2023-04-21": "Eid-ul-Fitr",
    "2023-04-22": "Ram Navami",
    "2023-06-29": "Eid-ul-Adha",
    "2023-08-30": "Raksha Bandhan",
    "2023-09-19": "Ganesh Chaturthi",
    "2023-10-24": "Dussehra",
    "2023-11-14": "Bhai Dooj",
    "2023-12-25": "Christmas"
}

# Converting dictionary to DataFrame
df = pd.DataFrame(list(festival_data.items()), columns=["Date", "Festival Name"])

# Ensure the directory exists
os.makedirs("data", exist_ok=True)

# Saving to CSV file
csv_filename = "data/festivals.csv"
df.to_csv(csv_filename, index=False)

csv_filename


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

# ---------------------------
# Configuration and constants (updated)
# ---------------------------
ATM_ID = "ATM_013" 
AVG_DAILY_WITHDRAWALS = 27
AVG_DAILY_INQUIRIES   = 1.35

ATM_MAX_CASH = 1200000
REFILL_THRESHOLD = 10000

# Updated monthly weighting factors.
# Updated monthly weighting factors (unchanged)
monthly_weights = {
    1: 1.19,  2: 1.1,  3: 2.99,  4: 1.62,  5: 1.1,  6: 1.44,
    7: 1.39,  8: 1.77,  9: 1.45, 10: 1.8, 11: 3.2, 12: 1.44
}

# Updated salary day multiplier range (unchanged)
SALARY_DATE_MULTIPLIER = (1.4, 1.75)

# Updated festival spikes for year 2023; note the dates have been revised.
festival_data = {
    "2023-03-08": "Holi",
    "2023-11-12": "Diwali",
    "2023-01-14": "Makar Sankranti",
    "2023-02-18": "Maha Shivratri",
    "2023-04-21": "Eid-ul-Fitr",
    "2023-04-22": "Ram Navami",
    "2023-06-29": "Eid-ul-Adha",
    "2023-08-30": "Raksha Bandhan",
    "2023-09-19": "Ganesh Chaturthi",
    "2023-10-24": "Dussehra",
    "2023-11-14": "Bhai Dooj",
    "2023-12-25": "Christmas"
}

# Hourly weights for 24 hours; these weights are later normalized to probabilities.
raw_hourly_weights = [
    2, 1, 0.5, 0.5, 0.5, 1, 3, 5, 6, 6,
    6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 7, 5, 4, 3
]
total_raw = sum(raw_hourly_weights)
hourly_probs = [w / total_raw for w in raw_hourly_weights]

# ---------------------------
# Sampling helper functions
# ---------------------------
def sample_refill_delay():
    """
    Return a random delay (in days) between 1 and 3.
    """
    return np.random.randint(1, 4)

def sample_cluster_delay():
    """
    Sample a delay (in minutes) for a clustered additional transaction.
    Uses an exponential distribution with a mean of 3 minutes.
    """
    delay = np.random.exponential(scale=3)
    return max(1, int(math.ceil(delay)))

def sample_withdrawal_amount(features):
    """
    Sample a withdrawal amount based on day type.
    
    • If it's a festival day → amount is uniformly chosen between 5,000 and 10,000.
    • Else if it's Saturday or Sunday → amount between 10,000 and 20,000.
    • Else if it's a salary day (and not weekend/festival) → amount between 7,000 and 20,000.
    • Otherwise (default) → amount between 10,000 and 20,000.
    
    The returned amount is rounded to the nearest 100.
    """
    if features["festival"]:
        low, high = 5000, 10000
    elif features["day_of_week"] in ("Saturday", "Sunday"):
        low, high = 10000, 20000
    elif features["salary_day"]:
        low, high = 7000, 20000
    else:
        low, high = 10000, 20000

    amount = np.random.randint(low, high + 1)
    return int(round(amount / 100.0) * 100)

# ---------------------------
# Feature engineering functions
# ---------------------------
def build_feature_vector(current_date):
    """
    Build a feature vector for the day with:
      - Festival indicator and its multiplier (if today's a festival).
      - Salary day indicator (day 1 to 5).
      - Weather conditions (rainy, coldest day, hot day) based on fixed rules.
      - Weekend flag and day-of-week name.
      - Combined event label.
    """
    features = {"date": current_date.date()}
    festival_indicator = ""
    festival_multiplier = 2.8  # default if needed
    
    # Check if today is a festival.
    for fest_name, (fest_date_str, multipliers) in festival_spikes.items():
        fest_date = datetime.strptime(fest_date_str, "%Y-%m-%d").date()
        if current_date.date() == fest_date:
            festival_indicator = fest_name
            # For multiple multipliers, we take the maximum.
            festival_multiplier = max(multipliers)
            break
    features["festival"] = festival_indicator
    features["festival_multiplier"] = festival_multiplier
    features["salary_day"] = (current_date.day in range(1, 6))
    # Adjust monthly weights considering weather impacts
    # Hypothetical base monthly weights (from your earlier configuration)

# Hypothetical weather effect multipliers for Lucknow in 2023
weather_impact_factors = {
    1: 1.00,  # January
    2: 1.00,  # February
    3: 0.95,  # March
    4: 0.90,  # April
    5: 0.80,  # May
    6: 0.85,  # June
    7: 0.70,  # July
    8: 0.85,  # August
    9: 0.90,  # September
    10: 1.00, # October
    11: 1.05, # November
    12: 1.05  # December
}

# Adjusted monthly weight factoring weather impact
adjusted_monthly_weights = {
    month: monthly_weights[month] * weather_impact_factors[month]
    for month in monthly_weights
}

print("Adjusted Monthly Weights considering Weather Impacts for 2023 in Lucknow:")
for month in sorted(adjusted_monthly_weights):
    print(f"Month {month}: {adjusted_monthly_weights[month]:.2f}")

def build_feature_vector(current_date):
    """
    Build a feature vector for the day with:
      - Festival indicator and its multiplier (if today's a festival).
      - Salary day indicator (day 1 to 5).
      - Weather conditions (rainy, coldest day, hot day) based on fixed rules.
      - Weekend flag and day-of-week name.
      - Combined event label.
    """
    features = {"date": current_date.date()}
    festival_indicator = ""
    festival_multiplier = 2.8  # default if needed
    
    # Check if today is a festival.
    for fest_name, (fest_date_str, multipliers) in festival_spikes.items():
        fest_date = datetime.strptime(fest_date_str, "%Y-%m-%d").date()
        if current_date.date() == fest_date:
            festival_indicator = fest_name
            # For multiple multipliers, we take the maximum.
            festival_multiplier = max(multipliers)
            break
    features["festival"] = festival_indicator
    features["festival_multiplier"] = festival_multiplier
    features["salary_day"] = (current_date.day in range(1, 6))
    
    weather = ""
    # Hard-coded rainy days.
    if (current_date.month, current_date.day) in {(7, 7), (8, 1), (8, 7), (8, 22), (8, 28)}:
        weather = "rainy"
    elif current_date.month == 12 and current_date.day >= 20:
        weather = "coldest day"
    # Selected hot days.
    elif (current_date.month == 6 and current_date.day in [6, 7, 8]) or (current_date.month == 7 and current_date.day in [6, 8]):
        weather = "hot day"
    features["weather"] = weather

    weekday = current_date.weekday()  # Monday=0, Sunday=6
    features["weekend"] = (weekday >= 5)
    day_of_week = current_date.strftime("%A")
    features["day_of_week"] = day_of_week

    if festival_indicator and weather:
        features["event_label"] = festival_indicator + ", " + weather
    elif festival_indicator:
        features["event_label"] = festival_indicator
    elif weather:
        features["event_label"] = weather
    else:
        features["event_label"] = ""
    
    return features

def predict_daily_demand(features):
    """
    Calculate an effective daily demand multiplier based on:
      - Festival and/or salary day (if both, multiply festival multiplier with a random salary factor).
      - Weather modifiers (rainy/cold days reduce demand; hot days reduce a bit too).
      - Weekend multipliers: Saturday ×1.3 and Sunday ×1.65.
    """
    if features["festival"] and features["salary_day"]:
        base_multiplier = features["festival_multiplier"] * np.random.uniform(SALARY_DATE_MULTIPLIER[0], SALARY_DATE_MULTIPLIER[1])
    elif features["festival"]:
        base_multiplier = features["festival_multiplier"]
    elif features["salary_day"]:
        base_multiplier = np.random.uniform(SALARY_DATE_MULTIPLIER[0], SALARY_DATE_MULTIPLIER[1])
    else:
        base_multiplier = 1.0

    if features["weather"] in ("rainy", "coldest day"):
        weather_modifier = 0.7
    elif features["weather"] == "hot day":
        weather_modifier = 0.8
    else:
        weather_modifier = 1.0

    effective_multiplier = base_multiplier * weather_modifier

    if features["day_of_week"] == "Saturday":
        effective_multiplier *= 1.3
    elif features["day_of_week"] == "Sunday":
        effective_multiplier *= 1.65

    return effective_multiplier

# ---------------------------
# Simulation state variables
# ---------------------------
remaining_cash = ATM_MAX_CASH
pending_refill_date = None  # Holds a datetime when a refill is scheduled

start_date_obj = datetime(2023, 1, 1)
end_date_obj   = datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date_obj, end=end_date_obj)

all_events = []  # List to store all transaction events

# ---------------------------
# Simulation main loop: iterate day by day
# ---------------------------
for current_date in date_range:
    
    # Process any scheduled refill event if due.
    if pending_refill_date is not None and current_date.date() >= pending_refill_date.date():
        event_hour = np.random.choice(range(24), p=hourly_probs)
        minute = np.random.randint(0, 60)
        second = np.random.randint(0, 60)
        event_time = f"{event_hour:02d}:{minute:02d}:{second:02d}"
        refill_event = [
            ATM_ID,
            current_date.date(),
            current_date.strftime('%A'),
            event_time,
            "Refill",
            0,                    # No withdrawal amount
            ATM_MAX_CASH,         # Cash resets to maximum
            str(current_date.date()),  # Refill Scheduled date written here
            "Refilled",
            None                  # No festival info for refill event
        ]
        all_events.append(refill_event)
        remaining_cash = ATM_MAX_CASH
        pending_refill_date = None

    # Build daily feature vector.
    features = build_feature_vector(current_date)
    day_of_week = features["day_of_week"]
    
    # Calculate daily demand.
    effective_multiplier = predict_daily_demand(features)
    daily_weight = monthly_weights[current_date.month] * effective_multiplier
    variation_factor = np.random.uniform(0.7, 1.3)
    
    # Determine the number of withdrawals and inquiries.
    num_withdrawals = np.random.poisson(lam=AVG_DAILY_WITHDRAWALS * daily_weight * variation_factor)
    num_inquiries   = np.random.poisson(lam=AVG_DAILY_INQUIRIES * daily_weight * variation_factor)
    
    events_list = []
    
    # Helper function to generate a random event time.
    def generate_event_time():
        hour = np.random.choice(range(24), p=hourly_probs)
        minute = np.random.randint(0, 60)
        second = np.random.randint(0, 60)
        return f"{hour:02d}:{minute:02d}:{second:02d}"
    
    # Generate withdrawal events.
    for i in range(num_withdrawals):
        event_time = generate_event_time()
        amount = sample_withdrawal_amount(features)
        events_list.append({
            "time_str": event_time,
            "type": "Withdrawal",
            "amount": amount,
            "festival": features["festival"]
        })
        # Optionally produce a clustered withdrawal shortly after.
        if np.random.rand() < 0.2:
            delay_minutes = sample_cluster_delay()
            base_hour, base_minute, base_second = map(int, event_time.split(":"))
            base_total = base_hour * 60 + base_minute
            new_total = base_total + delay_minutes
            if new_total < 24 * 60:  # Ensure within same day.
                new_hour = new_total // 60
                new_minute = new_total % 60
                new_time = f"{new_hour:02d}:{new_minute:02d}:{base_second:02d}"
                events_list.append({
                    "time_str": new_time,
                    "type": "Withdrawal",
                    "amount": sample_withdrawal_amount(features),
                    "festival": features["festival"]
                })
                
    # Generate inquiry events.
    for i in range(num_inquiries):
        event_time = generate_event_time()
        events_list.append({
            "time_str": event_time,
            "type": "Inquiry",
            "amount": 0,
            "festival": features["festival"]
        })
        
    # Sort events by time so they are processed chronologically.
    events_list.sort(key=lambda x: x["time_str"])
    
    # Process each event.
    for event in events_list:
        if event["type"] == "Withdrawal":
            # In the low-cash window, all withdrawals are declined.
            if remaining_cash < REFILL_THRESHOLD:
                status = "Declined"
                if pending_refill_date is None:
                    pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                refill_info = str(pending_refill_date.date())
            else:
                if remaining_cash >= event["amount"]:
                    remaining_cash -= event["amount"]
                    status = "Completed"
                    # If cash falls below threshold, schedule a refill if none exists.
                    if remaining_cash < REFILL_THRESHOLD and pending_refill_date is None:
                        pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                        refill_info = str(pending_refill_date.date())
                    else:
                        refill_info = ""
                else:
                    status = "Declined"
                    if remaining_cash < REFILL_THRESHOLD and pending_refill_date is None:
                        pending_refill_date = current_date + timedelta(days=sample_refill_delay())
                    refill_info = str(pending_refill_date.date()) if pending_refill_date is not None else ""
    
            processed_event = [
                ATM_ID,
                current_date.date(),
                day_of_week,
                event["time_str"],
                event["type"],
                event["amount"],
                remaining_cash,
                refill_info,  # The scheduled refill date is recorded here.
                status,
                event["festival"]
            ]
            all_events.append(processed_event)
            
        elif event["type"] == "Inquiry":
            processed_event = [
                ATM_ID,
                current_date.date(),
                day_of_week,
                event["time_str"],
                event["type"],
                0,
                remaining_cash,
                "",   # No refill info for inquiries.
                "Completed",
                event["festival"]
            ]
            all_events.append(processed_event)

# ---------------------------
# Create a DataFrame and save to CSV.
# ---------------------------
df_transactions = pd.DataFrame(all_events, columns=[
    "ATM_ID", "Date", "Day", "Time", "Transaction_Type",
    "Transaction_Amount", "Remaining_Cash", "Cash_Refill",
    "Transaction_Status", "Festival"
])
output_file = "2023ATM_013_withdrawal.csv"
df_transactions.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")13
